# [Superdense Coding Code used from IBM's Qiskit Tutorials](https://qiskit.org/textbook/ch-algorithms/superdense-coding.html)

Code from source modified

In [1]:
import unittest

import hypothesis.strategies as st
from hypothesis import given, settings, note

from qiskit import QuantumCircuit
from qiskit import Aer, execute
from qiskit.circuit.library import Barrier, XGate, HGate, CXGate, ZGate, Measure

In [2]:
def create_bell_pair():
    qc = QuantumCircuit(2)
    qc.h(1)
    qc.cx(1, 0)
    return qc

In [3]:
def encode_message(qc, qubit, msg):
    if len(msg) != 2 or not set([0,1]).issubset({0,1}):
        raise ValueError(f"message '{msg}' is invalid")
    if msg[1] == "1":
        qc.x(qubit)
    if msg[0] == "1":
        qc.z(qubit)
    return qc

In [4]:
def decode_message(qc):
    qc.cx(1, 0)
    qc.h(1)
    return qc

In [5]:
qc = create_bell_pair()

message = '11'

qc = encode_message(qc, 1, message)

qc = decode_message(qc)

qc.measure_all()

print(qc)

backend = Aer.get_backend('aer_simulator') 
job = execute(qc, backend, shots=1, memory=True)
readings = job.result().get_memory()
print(readings)

             ┌───┐          ┌───┐      ░ ┌─┐   
   q_0: ─────┤ X ├──────────┤ X ├──────░─┤M├───
        ┌───┐└─┬─┘┌───┐┌───┐└─┬─┘┌───┐ ░ └╥┘┌─┐
   q_1: ┤ H ├──■──┤ X ├┤ Z ├──■──┤ H ├─░──╫─┤M├
        └───┘     └───┘└───┘     └───┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════╩══╩═
                                          0  1 
['11']


### Validity Testing
Testing for the presence of gates is not exactly what we are looking for however, if the plan is to actually use validity testing for these circuits, there needs to be a better way.

For now consider only the postcondition testing...

Mutation testing caught all mutations without the validity testing anyway. 

In [19]:
@st.composite
def draw_message(draw):
    drawnInt = draw(st.integers(min_value=0, max_value=1))
    drawnInt2 = draw(st.integers(min_value=0, max_value=1))
    return(str(drawnInt)+str(drawnInt))

# def isValid(qc):
#     valid = True
#     for gate in qc:
#         if (not (isinstance(gate[0], Barrier) 
#             or isinstance(gate[0], XGate)
#             or isinstance(gate[0], HGate) 
#             or isinstance(gate[0], ZGate)
#             or isinstance(gate[0], Measure)
#             or isinstance(gate[0], CXGate))):
#                 valid = False
#     return valid

# def test_bell_state_is_valid():
#     assert(isValid(create_bell_pair()))

# @given(draw_message())
# @settings(deadline=None)
# def test_encode_message_is_valid(message):
#     assert(isValid(encode_message(create_bell_pair(), 1, message)))
    
# @given(draw_message())
# @settings(deadline=None)   
# def test_decode_message_is_valid(message):
    assert(isValid(decode_message(encode_message(create_bell_pair(), 1, message))))

### Normal Tests

In [20]:
# def test_bell_pair_returns_entangled_values():
#     qc = create_bell_pair()
#     qc.measure_all()
#     backend = Aer.get_backend('aer_simulator') 
#     job = execute(qc, backend, shots=1, memory=True)
#     readings = job.result().get_memory()
#     assert(readings == ['11'] or readings == ['00'])
    
# @given(st.integers(min_value=0, max_value=100))
# @settings(deadline=None)
# def test_bell_pair_returns_entangled_values_even_after_hadamard_applied(x):
#     ## we dont need the given integer, we are just using it to perform extra tests
#     ## as we will not always fail with incorrect code
#     qc = create_bell_pair()
#     qc.h(0)
#     qc.h(1)
#     qc.measure_all()
#     backend = Aer.get_backend('aer_simulator') 
#     job = execute(qc, backend, shots=1, memory=True)
#     readings = job.result().get_memory()
#     note(readings)
#     assert(readings == ['11'] or readings == ['00']) 

### Postcondition Property tests

In [21]:
@given(draw_message())
@settings(deadline=None)
def test_encode_message_returns_only_sets_of_bell_pair_values(message):
    qc = create_bell_pair()
    qc = encode_message(qc, 1, message)
    qc.measure_all()
    note(message)
    backend = Aer.get_backend('aer_simulator') 
    job = execute(qc, backend, shots=1000, memory=True)
    readings = job.result().get_memory()
    note(readings)
    note(not(set(readings) == set(['11','00']) and set(readings) == set(['01','10'])))
    note(set(readings) == set(['11','00']) or set(readings) == set(['01','10']))
    assert(not(set(readings) == set(['11','00']) and set(readings) == set(['01','10'])))
    assert(set(readings) == set(['11','00']) or set(readings) == set(['01','10']))
    
    
@given(draw_message())
def test_decode_message_equal_to_encode_message(message):
    note(message)
    qc = create_bell_pair()
    qc = encode_message(qc, 1, message)
    qc = decode_message(qc)
    qc.measure_all()
    backend = Aer.get_backend('aer_simulator') 
    job = execute(qc, backend, shots=1, memory=True)
    readings = job.result().get_memory()
    note(readings)
    assert(readings == [message])

In [22]:
if __name__ == '__main__':
    #test_bell_state_is_valid()
    #test_encode_message_is_valid()
    #test_decode_message_is_valid()
    #test_bell_pair_returns_entangled_values()
    #test_bell_pair_returns_entangled_values_even_after_hadamard_applied()
    test_encode_message_returns_only_sets_of_bell_pair_values()
    test_decode_message_equal_to_encode_message()

['00', '00', '00', '00', '11', '11', '11', '00', '11', '00', '11', '00', '11', '00', '11', '00', '11', '00', '11', '11', '00', '00', '00', '11', '00', '00', '00', '11', '11', '11', '11', '00', '00', '11', '11', '11', '00', '00', '00', '00', '00', '11', '00', '11', '11', '11', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '11', '00', '00', '11', '11', '11', '00', '11', '00', '11', '11', '00', '11', '00', '11', '11', '11', '11', '11', '00', '00', '00', '00', '00', '11', '11', '00', '00', '00', '00', '11', '00', '00', '11', '00', '11', '00', '00', '11', '11', '11', '00', '11', '11', '00', '11', '11', '00', '11', '11', '11', '00', '11', '00', '11', '11', '00', '11', '00', '00', '00', '00', '11', '00', '11', '00', '11', '11', '00', '11', '11', '00', '00', '11', '11', '00', '11', '11', '11', '00', '00', '11', '00', '11', '11', '11', '00', '11', '00', '11', '00', '11', '00', '11', '00', '11', '11', '11', '11', '11', '11', '00', '11', '00', '11', '00', '11', '11', '11', '00